**1. Introducción**

El siguiente ejemplo realiza la multiplicación entre un numero (pasado por parámetro) y una matriz[4] utilizando GPGPU. Además, se pasa por parametro el tamaño de las filas y columnas que debe tener la matriz.

Utilizando lenguaje Python[1] en Google Colab[2]

**2. Armado del Ambiente**

Instala en el cuaderno el módulo CUDA de Python. [3]

In [ ]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 5.7MB/s 
     |████████████████████████████████| 71kB 10.2MB/s 
     |████████████████████████████████| 81kB 9.1MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=620899 sha256=6ed11cfa335cc911037a17bc59922dec9c433c46a3ea4360bb72d9cc2858fde6
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=303ac6432fbddf32f3011752be6211726ce7e96b6c1d4a9e3bbe230ef53cc50b
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


**3. Desarrollo**

In [ ]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

TamX =   3#@param {type: "number"}
TamY =   4#@param {type: "number"}
Escalar = 5000#@param {type: "number"}

import pycuda.driver as cuda
import pycuda.autoinit
from   pycuda.compiler import SourceModule


from datetime import datetime

tiempo_total = datetime.now()

import numpy

try:
  if TamX <= 0 or TamY <= 0 or Escalar <= 0:
    print("Ingrese los parametros correctamente. (TamX, TamY y Escalar deben ser mayores a 0")
  else:
    # --------------------------------------------
    # Definición de función que transforma el tiempo en  milisegundos 
    tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

    # CPU - Defino la memoria de las matrices en cpu.
    matriz = numpy.random.random((TamX,TamY))
    matriz = matriz.astype(numpy.float32())

    matrizResultado = numpy.random.random((TamX,TamY))
    matrizResultado = matriz.astype(numpy.float32())

    # CPU - reservo la memoria GPU.
    matrizGPU = cuda.mem_alloc(matriz.nbytes)
    matrizResultadoGPU = cuda.mem_alloc(matrizResultado.nbytes)

    # GPU - Copio la memoria al GPU.
    cuda.memcpy_htod(matrizGPU,matriz)
    cuda.memcpy_htod(matrizResultadoGPU,matrizResultado)

    # CPU - Informo el resutlado.
    print( "------------------------------------")
    print( "Matriz Original: " )
    print( matriz )
    print( "------------------------------------")
    #CPU - Defino la funcion kernel que ejecutará en GPU
    module = SourceModule("""
    __global__ void kernel_matriz(int filas, int columnas, float *matriz ,float *matrizResultado, int Escalar)
    {
        int idx = threadIdx.x + blockIdx.x*blockDim.x;
        int idy = threadIdx.y + blockIdx.y*blockDim.y;

        if(idx<filas && idy<columnas)
        {
          matrizResultado[idy*filas + idx] = Escalar * matriz[idx*columnas + idy];
        }
    }

    """)

    kernel = module.get_function("kernel_matriz")

    dim_hilo_x = 16
    dim_bloque_x = numpy.int( (TamX+dim_hilo_x-1) / dim_hilo_x )

    dim_hilo_y = 16
    dim_bloque_y = numpy.int( (TamY+dim_hilo_y-1) / dim_hilo_y )

    print( "Thread x: ", dim_hilo_x, ", Bloque x:", dim_bloque_x )
    print( "Thread y: ", dim_hilo_y, ", Bloque y:", dim_bloque_y )

    tiempo_kernel = datetime.now()


    kernel( numpy.int32(TamX), numpy.int32(TamY), matrizGPU, matrizResultadoGPU, numpy.int32(Escalar), block=( dim_hilo_x, dim_hilo_y, 1 ),grid=(dim_bloque_x, dim_hilo_y,1) )

    tiempo_kernel = datetime.now() - tiempo_kernel

    cuda.memcpy_dtoh(matrizResultado,matrizResultadoGPU)

    tiempo_total = datetime.now() - tiempo_total
    print( "------------------------------------")
    print("Matriz Resultado: ")
    print(matrizResultado)
    print( "------------------------------------")
    print("Tiempo TOTAL: ", tiempo_en_ms( tiempo_total ), "[ms]" )
    print("Tiempo GPU  : ", tiempo_en_ms( tiempo_kernel ), "[ms]" )
except Exception as exception:
  print("Ha ocurrido una excepcion: ", exception)

------------------------------------
Matriz Original: 
[[0.09634965 0.37397656 0.52419794 0.77885205]
 [0.91160655 0.7586633  0.9115434  0.5276657 ]
 [0.56648964 0.35144705 0.99532706 0.2814266 ]]
------------------------------------
Thread x:  16 , Bloque x: 1
Thread y:  16 , Bloque y: 1
------------------------------------
Matriz Resultado: 
[[ 481.74826 4558.0327  2832.4482  1869.8828 ]
 [3793.3164  1757.2352  2620.9897  4557.7173 ]
 [4976.6353  3894.2603  2638.3284  1407.133  ]]
------------------------------------
Tiempo TOTAL:  5.216 [ms]
Tiempo GPU  :  0.12 [ms]


**4. Tabla de Pasos**

Procesador	| Función	| Detalle
------------|---------|--------
CPU	| @param	| Lectura del tamaño de vectores desde Colab.
CPU	| import	| Importa los módulos para funcionar.
CPU	| datetime.now()	| Toma el tiempo actual.
CPU | if | Verifico los parametros
CPU	| numpy.random.random((TamX,TamY))	| Inicializa las matrices.
**GPU**	| cuda.mem_alloc()	| Reserva la memoria en GPU.
**GPU**	| cuda.memcpy_htod()	| Copia las memorias desde el CPU al GPU.
CPU	| SourceModule()	| Define el código del kernel
CPU	| module.get_function()	| Genera la función del kernel GPU
CPU	| dim_hilo_x, dim_hilo_y	| Calculo las dimensiones para la ejecucion en 2D
**GPU**	| kernel()	| Ejecuta el kernel en GPU
CPU	| cuda.memcpy_dtoh( )	| Copia el resultado desde GPU memoria Y a CPU memoria R.
CPU	| print()	| Informo la matriz resultante.

**5. Conclusión**

Continuando con las pruebas realizadas en CPU, replique lo mismo en GPU.

Ante una matriz 500x500 los valores en general bajan mucho

Tiempo TOTAL:  14,27 [ms] - Tiempo GPU  :  0,094 [ms]

Recordando que al ejecutarse en CPU, el tiempo TOTAL de ejecución fue mucho mayor Tiempo Total: 820,462 [ms].

En estos casos se puede ver que ante una matriz de tamaño grande, es recomendable utilizarla con GPU.

En cuanto a la matriz de 3x4 los valores son los opuestos, a la conclusión que se llega en este caso es la recomendación de ejecutar en CPU si la matriz es pequeña.

Tiempo TOTAL:  5,216 [ms] - Tiempo GPU  :  0,12 [ms]

Tiempo Total en CPU: 3,841 [ms]

**6. Bibliografía**

[1] Introducción a Python [Link](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb)

[2] Introducción a Colab [Link](https://www.youtube.com/watch?v=ICJP_ukNSQ0)

[3] Documentación PYCUDA [Link](https://documen.tician.de/pycuda/)

[4] Algoritmo Matriz * Escalar [Link](https://es.khanacademy.org/math/precalculus/x9e81a4f98389efdf:matrices/x9e81a4f98389efdf:multiplying-matrices-by-scalars/a/multiplying-matrices-by-scalars)